In [3]:
API_KEY="695bba0b702f4443ac5ee2022411804f"

import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
baseURL = "https://newsapi.org/v2/everything?"
total_requests=2
verbose=True

topic_1 = 'medical cost USA'

URLpost_1 = {'apiKey': API_KEY,
            'q': '+'+topic_1,
            'sortBy': 'relevancy',
            'totalRequests': 1}

response_1 = requests.get(baseURL, URLpost_1) #request data from the server
response_1 = response_1.json() #extract txt data from request into json

# #GET TIMESTAMP FOR PULL REQUEST
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d-H%H-M%M-S%S")

# SAVE TO FILE 
with open(timestamp+'-newapi-medicine-raw-data.json', 'w') as outfile:
    json.dump(response_1, outfile, indent=4)
 
 

def string_cleaner(input_string):
    try: 
        out=re.sub(r"""
                    [,.;@#?!&$-]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    input_string, flags=re.VERBOSE)

        #REPLACE SELECT CHARACTERS WITH NOTHING
        out = re.sub('[’.]+', '', input_string)

        #ELIMINATE DUPLICATE WHITESPACES USING WILDCARDS
        out = re.sub(r'\s+', ' ', out)

        #CONVERT TO LOWER CASE
        out=out.lower()
    except:
        print("ERROR")
        out=''
    return out
article_list_1 =response_1['articles']   #list of dictionaries for each article
article_keys_1=article_list_1[0].keys()
print("AVAILABLE KEYS:")
print(article_keys_1)
index=0
cleaned_data_1=[];  

for article in article_list_1:
    tmp=[]
    if(verbose):
        print("#------------------------------------------")
        print("#",index)
        print("#------------------------------------------")

    for key in article_keys_1:
        if(verbose):
            print("----------------")
            print(key)
            print(article[key])
            print("----------------")

        if(key=='source'):
            src=string_cleaner(article[key]['name'])
            tmp.append(src) 

        if(key=='author'):
            author=string_cleaner(article[key])
            #ERROR CHECK (SOMETIMES AUTHOR IS SAME AS PUBLICATION)
            if(src in author): 
                print(" AUTHOR ERROR:",author);author='NA'
            tmp.append(author)

        if(key=='title'):
            tmp.append(string_cleaner(article[key]))

        if(key=='description'):
            tmp.append(string_cleaner(article[key]))

        # if(key=='content'):
        #     tmp.append(string_cleaner(article[key]))

        if(key=='publishedAt'):
            #DEFINE DATA PATERN FOR RE TO CHECK  .* --> wildcard
            ref = re.compile('.*-.*-.*T.*:.*:.*Z')
            date=article[key]
            if(not ref.match(date)):
                print(" DATE ERROR:",date); date="NA"
            tmp.append(date)

    cleaned_data_1.append(tmp)
    index+=1   


df_1 = pd.DataFrame(cleaned_data_1, columns = ['src', 'author', 'title', 'description', 'url'])
df_1 = df_1[['title', 'description']]
df_1.to_csv('medicine-cleaned.csv', index=False)
df_1.head()

AVAILABLE KEYS:
dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
#------------------------------------------
# 0
#------------------------------------------
----------------
source
{'id': None, 'name': 'Eurogamer.net'}
----------------
----------------
author
Geoffrey Bunting
----------------
----------------
title
A round-up of the talks from GAconf USA 2023
----------------
----------------
description
GAconf returned to the US this week with a series of accessibility talks hosted at the Archer Hotel in Redmond, Washington. Virtual attendance was available through Zoom and YouTube. Talks ranged from those on personal experiences of disability and accessibil…
----------------
----------------
url
https://www.eurogamer.net/a-round-up-of-the-talks-from-gaconf-usa-2023
----------------
----------------
urlToImage
https://assetsio.reedpopcdn.com/FORZA-SITE_QkZV9pp.jpg?width=1200&height=630&fit=crop&enable=upscale&auto=webp
------------

,title,description
0,a round-up of the talks from gaconf usa 2023,gaconf returned to the us this week with a ser...
1,american can prevent (and control) type 2 diab...,usa today's health team spoke with scores of e...
2,the steep cost of type 2: when diabetes dragge...,the nation's disjointed and confusing health c...
3,a hidden system of exploitation underpins us h...,this series was produced in partnership with t...
4,the childcare cliff: $122 billion dollar crisi...,the childcare crisis in america; the harsh tru...
